# Random Strategy

Start off with a random strategy of choosing which transactions and cards to open

In [1]:
import pandas as pd
import numpy as np
import json
from Card import Card

In [11]:
# read in data
transactions = pd.read_csv('transactions.csv')
with open('card_options.json', 'r') as f:
    card_options = json.load(f)

In [ ]:
# Meta code
# build class of user
# initiatlize user
# go through transactions month by month, starting with card opening, and then with trans categories
# randomly choose which card to open (choose arbitrary starting point)
# randomly choose which card to put spend on
# keep track of rewards (spend) as well as total spend and fees